# Attention

| 역할 | 설명 |
|------|------|
| **Query (Q)** | 내가 "무엇을 알고 싶은지"를 표현하는 질문 벡터 |
| **Key (K)**   | 각 단어가 "어떤 정보"를 갖고 있는지 나타내는 기준 벡터 |
| **Value (V)** | 실제 그 단어가 가진 정보 벡터 |


In [1]:
import torch
import torch.nn as nn

import torch.nn.functional as F

### 어텐션 가중치 계산

In [2]:
def attention(query, key, value):
    # 1. 어텐션 스코어 계산 (Query와 Key의 내적)
    scores = torch.matmul(query, key.transpose(-2, -1))
    print('Attention Score Shape:', scores.shape)

    # 2. Softmax를 통해 어텐션 스코어를 확률로 변환 -> 가중치로 사용
    attention_weights = F.softmax(scores, dim=-1)
    print('Attention weights shape:', attention_weights.shape)

    # 3. 어텐션 밸류 계산 (Value 적용 => 최종 Context vector 계산)
    context_vector = torch.matmul(attention_weights, value)
    print('Context vector shape:', context_vector.shape)

    return context_vector

In [3]:
# 토큰화 및 임베딩 결과 예시
vocab = {
    "나는": 0,
    "학교에": 1, 
    "간다": 2,
    "<pad>": 3
}

vocab_size = len(vocab)
EMBEDDING_DIM = 4

In [4]:
# 입력 문장
inputs = ["나는", "학교에", "간다"]
inputs_ids = torch.tensor([[vocab[word] for word in inputs]]) # (1, 3)으로 배치차원을 맞춤

In [5]:
# 1. 임베딩 적용
embedding_layer = nn.Embedding(vocab_size, EMBEDDING_DIM) 
inputs_embedded = embedding_layer(inputs_ids)
# print(inputs_embedded.shape)    # (1, 3, 4) -> 배치 차원, 시퀀스 길이, 임베딩 차원 

# 2. 선형 변환 -> Query, Key, Value 생성
HIDDEN_DIM = 4
W_query = nn.Linear(EMBEDDING_DIM, HIDDEN_DIM)
W_key = nn.Linear(EMBEDDING_DIM, HIDDEN_DIM)
W_value = nn.Linear(EMBEDDING_DIM, HIDDEN_DIM)

input_query = W_query(inputs_embedded)
input_key = W_key(inputs_embedded)
input_value = W_value(inputs_embedded)

print(input_query.shape, input_key.shape, input_value.shape)    # 모두 (1, 3, 4)  (배치 차원, 시퀀스 길이, 히든 차원)



torch.Size([1, 3, 4]) torch.Size([1, 3, 4]) torch.Size([1, 3, 4])


In [6]:
context_vector = attention(input_query, input_key, input_value)
context_vector

Attention Score Shape: torch.Size([1, 3, 3])
Attention weights shape: torch.Size([1, 3, 3])
Context vector shape: torch.Size([1, 3, 4])


tensor([[[-0.4807,  0.2982, -0.2995, -0.3445],
         [-0.4408,  0.2936, -0.1907, -0.2689],
         [-0.3736,  0.3162, -0.2247, -0.3861]]], grad_fn=<UnsafeViewBackward0>)

### seq2seq 모델에 어텐션 추가

In [7]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size) # query와 key를 concat한 후 선형변환 (hidden_size * 2 -> hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))  # 어텐션의 가중치 벡터 
        
    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.shape[1]
        hidden_expanded = hidden.unsqueeze(1).repeat(1, seq_len, 1) #(batch_size, seq_len, hidden_size)로 확장 (1, seq_len, 1) : 1의 위치는 유지해주면서 seq_len 확장
        energy = torch.tanh(self.attn(torch.cat((hidden_expanded, encoder_outputs), dim=2))) # 디코더의 현재 상태와 인코더의 출력을 연결
        attention_scores = torch.sum(self.v * energy, dim=2)    # 현재 상태에 가중치를 곱해 합계를 구함 -> attention score
        attention_weights = F.softmax(attention_scores, dim=1)  # 어텐션 가중치 계산
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)  # 최종 context vector계산 
        return context_vector, attention_weights

In [8]:
class Seq2SeqWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2SeqWithAttention, self).__init__()
        self.encoder = nn.GRU(input_dim, hidden_dim, batch_first=True)  # 인코더 GRU
        self.decoder = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.decoder_input_transform = nn.Linear(input_dim, hidden_dim)  # 디코더 입력 변환 (인코더의 출력과 같은 차원으로 변환)
            
    def forward(self, encoder_input, decoder_input):
        encoder_outputs, hidden = self.encoder(encoder_input)
        context_vector, _ = self.attention(hidden[-1], encoder_outputs) # 인코더의 마지막 hidden state와 인코더의 출력을 사용하여 context vector 계산
        decoder_input_ = self.decoder_input_transform(decoder_input)
        output, _ = self.decoder(decoder_input_, hidden)
        combined = torch.cat((output, context_vector.unsqueeze(1)), dim=2)
        return self.fc(combined)  # 최종 출력

In [9]:
batch_size = 1
seq_len = 5
input_dim = 10
hidden_dim = 20
output_dim = 15
encoder_input = torch.randn(batch_size, seq_len, input_dim)
decoder_input = torch.randn(batch_size, 1, input_dim)

model = Seq2SeqWithAttention(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
result = model(encoder_input, decoder_input)
print(result)

tensor([[[ 0.1092,  0.0646,  0.1446, -0.0725,  0.0600, -0.1632, -0.0874,
          -0.0095,  0.1874, -0.0808,  0.1002, -0.2474, -0.0165,  0.2120,
          -0.0896]]], grad_fn=<ViewBackward0>)
